In [8]:
%load_ext autoreload
%autoreload 2
import numpy as np


1. sharing internet on the laocal machine

````
    ifconfig
````

2. scanning the local network to find the raspberry π

````
    nmap -T4 -sP 192.168.2.0/24
````

3. what I used to clone the repository on the π

````
  686  pip install pyserial
  687  vim .ssh/id_rsa.pub
  688  vim .ssh/id_rsa
  689  chmod 0600 .ssh/id_rsa
  690  git clone git@git.framasoft.org:laurentperrinet/elasticte.git
  691  cd elasticte/
  692  ls
  693  cat scenario_line_contraint.py
  694  python scenario_line_contraint.py  serial
  695  pip install -e .
  696  pip install --user -e .
  697  python scenario_line_contraint.py  serial
````

4. starting 

````

⇒  ssh pi@192.168.2.3
pi@192.168.2.3's password: tropic

Linux pielastic 3.18.7-v7+ #755 SMP PREEMPT Thu Feb 12 17:20:48 GMT 2015 armv7l

pi@pielastic ~ $ cd elasticte/

pi@pielastic ~/elasticte $ git pull # to update the code
Already up-to-date.

pi@pielastic ~/elasticte $ python scenario_line_contraint.py serial

````


## coordonées absolues

Prenons un cas  simple avec 12 pas possible et calculons en fonction de la position actuelle et de la position désirée la commande à envoyer aux moteurs

In [7]:
nb_pas = 12
position_present = 6
position_desired = 4

d_position = (position_desired - position_present + nb_pas//2 ) % nb_pas - nb_pas//2

print (d_position)

position_present = (position_present + d_position ) % nb_pas

print (position_present)


-2
4


on vérifie que ça marche pour toute une série de positions désirées et qu'on évite le demi tour:

In [13]:
nb_pas = 12
position_present = 2
position_desired = np.arange(nb_pas)

d_position = (position_desired - position_present + nb_pas//2 ) % nb_pas - nb_pas//2

print (d_position)

position_present = (position_present + d_position ) % nb_pas

print (position_present)


[-2 -1  0  1  2  3  4  5 -6 -5 -4 -3]
[ 0  1  2  3  4  5  6  7  8  9 10 11]


on vérifie aussi que ça marche pour toute une série de positions initiales et qu'on évite le demi tour:

In [14]:
nb_pas = 12
position_present = np.arange(nb_pas)
position_desired = 8

d_position = (position_desired - position_present + nb_pas//2 ) % nb_pas - nb_pas//2

print (d_position)

position_present = (position_present + d_position ) % nb_pas

print (position_present)


[-4 -5 -6  5  4  3  2  1  0 -1 -2 -3]
[8 8 8 8 8 8 8 8 8 8 8 8]


### arduino

coordonnées moteur + engrenage:
 - 1.8 deg par pas (soit 200 pas par tour) x 32 divisions de pas = 
 - demultiplication : pignon1= 14 dents, pignon2 = 60 dents


In [3]:
n_pas = 200. * 32. * 60 / 14
print(n_pas)

27428.571428571428


Donc, on a un nombre même pas entier mais c'est pas grave. C'est le nombre de pas à faire pour le moteur pour faire faire un tour à la lame. Dans un premier temps, on va considérer ce cercle comme des entiers entre 0 et n_pas. ça devrait pouvoir être envoyé sans probleme a chaque arduino comme un int16 (à vérifier - si on a de la marge, on peut faire un truc plus précis): 

In [5]:
2**16

65536

L'algo sur le arduino doit donc avoir cette partie pour convertir une donnée absolue reçue en donnée relative à envoyer aux moteurs:

In [15]:
nb_pas = 27428

position_present = 27424
position_desired = 4 # recu avec la lecture sur le port série

# quand on reçoit une valeur absolue on calcule
d_position = (position_desired - position_present + nb_pas//2 ) % nb_pas - nb_pas//2

# on envoie d_position aux moteurs

# on met à jour la position du moteur
position_present = (position_present + d_position ) % nb_pas

print(d_position, position_present)

8 4


## git


In [11]:
!git s
#!git add 2015-10-27\ élasticité\ r*

 M "2015-10-27 \303\251lasticit\303\251 expansion-r\303\251action diffusion.ipynb"
 M "2015-10-27 \303\251lasticit\303\251 expansion.ipynb"
 M "2015-10-27 \303\251lasticit\303\251 rapsberry pyserial.ipynb"
 M ../scenario_line_contraint.py
 M ../src/elasticite.py
?? ../.ipynb_checkpoints/
?? ../PLAN_PRINCIPE_PROTO/
?? ../Presentation-Dossier-Artistique/
?? ../TESTS_TRAME/
?? ../__temp_ipython__.png
?? ../files/elastic_fresnel.mp4
?? ../files/elastic_fresnel_gauche.mp4
?? ../files/elastic_fresnel_lateral.mp4
?? ../files/elastic_fresnel_pasapas.mp4
?? ../files/elastic_fresnel_rot.mp4
?? ../files/fresnel.mp4
?? "../files/grille_carr\303\251e.mp4"
?? ../files/lames_contraintes.mp4
?? ../files/lames_oscillantes.mp4
?? ../files/lames_oscillantes_fps5.mp4
?? "../notes-Laurent/2015-10-22_\303\251lasticit\303\251 LAMES-Dynamique_pdf.pdf"
?? .ipynb_checkpoints/
?? __temp__.mp4
?? __temp_ipython__.png
?? anim.gif
?? ../screenshot.png
?? ../src/elasticite.egg-info/


In [12]:
!git commit -am' on connecte le π + coordonnées absolues'

[master e8cbc37]  on connecte le π + simplifying things
 5 files changed, 131 insertions(+), 134 deletions(-)


In [13]:
! git push

To git@git.framasoft.org:laurentperrinet/elasticte.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to 'git@git.framasoft.org:laurentperrinet/elasticte.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
